# Calculate UMAP embeddings with different parameters

In [1]:
import os
import pandas as pd
import sys
import numpy as np
from pandas.core.common import flatten
import pickle
import umap
from scipy.stats import zscore
from pathlib import Path

In [2]:
wd = os.getcwd()

DF = os.path.join(os.path.sep, str(Path(wd).parents[0]), "data", "processed", "df_focal_reduced.pkl")
OUT_COORDS = os.path.join(os.path.sep, str(Path(wd).parents[0]), "data", "interim", "parameter_search", "umap_coords")
OUT_EVALS = os.path.join(os.path.sep, str(Path(wd).parents[0]), "data", "interim", "parameter_search", "umap_evals")

In [3]:
spec_df = pd.read_pickle(DF)
spec_df.shape

(6430, 28)

## Functions

In [4]:
def set_defaults():
    global input_type
    global preprocess_type 
    global metric_type
    global duration_method
    global min_dist
    global spread
    global n_neighbors
    global n_comps    
    
    input_type = 'melspecs'
    preprocess_type = 'zs'
    metric_type = 'manhattan'
    duration_method = 'pad'
    min_dist = 0
    spread = 1
    n_neighbors=15
    n_comps = 5

In [5]:
def get_param_string():
    param_combi = "_".join([str(x) for x in [preprocess_type, metric_type, duration_method,
                                         min_dist, spread, n_neighbors, n_comps, input_type]])
    return param_combi

In [6]:
def calc_umap(data, outname, metric='manhattan', min_dist=0, spread=1, n_neighbors=15, n_comps=5,n=5):
    
    for i in range(n):
        reducer = umap.UMAP(n_components = n_comps, 
                            min_dist=min_dist,
                            spread=spread,
                            n_neighbors=n_neighbors,
                            metric=metric)

        embedding = reducer.fit_transform(data) 
        np.savetxt(outname+'_'+str(i)+'.csv', embedding, delimiter=";")   

In [7]:
# Function that pads a spectrogram with zeros to a certain length
# Input: spectrogram (2D np array)
#        maximal length (Integer)
# Output: Padded spectrogram (2D np array)

def pad_spectro(spec,maxlen):
    padding = maxlen - spec.shape[1]
    z = np.zeros((spec.shape[0],padding))
    padded_spec=np.append(spec, z, axis=1)
    return padded_spec

def create_padded_data(specs):
    maxlen= np.max([spec.shape[1] for spec in specs])
    flattened_specs = [pad_spectro(spec, maxlen).flatten() for spec in specs]
    data = np.asarray(flattened_specs)
    return data

# Different input types

In [202]:
set_defaults()

input_types = ['amplispecs', 'freqspecs', 'melspecs', 'mfccs']
input_dict = dict(zip(input_types, ['ampli_spectrograms', 'freq_spectrograms', 'spectrograms','mfccs']))

In [203]:
for input_type in input_types:
    
    specs = spec_df[input_dict[input_type]]
    specs = [calc_zscore(s) for s in specs]
    data = create_padded_data(specs)
    
    outname = os.path.join(os.path.sep, OUT_COORDS, get_param_string())
    print(outname)
    calc_umap(data, outname)

/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_15_5_amplispecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_15_5_freqspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_15_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_15_5_mfccs


## Different preprocessing steps

In [8]:
import numba
from numba import jit

MEL_BINS_REMOVED_LOWER = 5
MEL_BINS_REMOVED_UPPER = 5
N_MELS = 40

@jit(nopython=True)
def calc_zscore(spec):
    mn = np.mean(spec)
    std = np.std(spec)
    for i in range(spec.shape[0]):
        for j in range(spec.shape[1]):
            spec[i,j] = (spec[i,j]-mn)/std
    return spec

@jit(nopython=True)
def preprocess_spec_numba_fl(spec):
    
    spec = spec[MEL_BINS_REMOVED_LOWER:(N_MELS-MEL_BINS_REMOVED_UPPER),:]
    spec = calc_zscore(spec)
    spec = np.where(spec < 0, 0, spec)
    
    return spec

@jit(nopython=True)
def preprocess_spec_numba(spec):
    
    spec = spec[MEL_BINS_REMOVED_LOWER:(N_MELS-MEL_BINS_REMOVED_UPPER),:]
    spec = calc_zscore(spec)
    spec = np.where(spec > 3, 3, spec)
    spec = np.where(spec < 0, 0, spec)
    
    return spec

In [42]:
specs = spec_df.spectrograms

preprocessed_specs = {'no': specs,
                     'zs' : [calc_zscore(s) for s in specs],
                     'zs-cu': [calc_zscore(s[MEL_BINS_REMOVED_LOWER:(N_MELS-MEL_BINS_REMOVED_UPPER),:]) for s in specs],
                     'zs-cu-fl': [preprocess_spec_numba_fl(s) for s in specs],
                     'zs-cu-fl-ce': [preprocess_spec_numba(s) for s in specs]}
preprocess_types = list(preprocessed_specs.keys())

In [43]:
set_defaults()

for preprocess_type in preprocess_types:
    specs = preprocessed_specs[preprocess_type]
    data = create_padded_data(specs)
    
    outname = os.path.join(os.path.sep, OUT_COORDS, get_param_string())
    print(outname)
    calc_umap(data, outname)

/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/no_euclidean_pad_0_1_15_5
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_euclidean_pad_0_1_15_5
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs-cu_euclidean_pad_0_1_15_5
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs-cu-fl_euclidean_pad_0_1_15_5
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs-cu-fl-ce_euclidean_pad_0_1_15_5


## Different ways to deal with variable duration

In [144]:
def pad_transform_spectro(spec,maxlen):
    
    flat_spec = spec.flatten()
    flat_spec.shape
    spec_l = flat_spec.shape[0]
    trans_spec = np.concatenate((np.asarray([spec_l]), flat_spec, np.asarray([999]*(maxlen-spec_l-1))))
    trans_spec = np.float64(trans_spec)
    
    return trans_spec

In [151]:
SPEC_SIZE = 40
MIN_OVERLAP = 0.9

@numba.njit()
def unpack_specs(a,b):   
    
    # len is encoded in first cell
    len_a = int(a[0])
    len_b = int(b[0])

    # reshape specs
    spec_a= np.reshape(a[1:len_a+1], (SPEC_SIZE, -1))
    spec_b= np.reshape(b[1:len_b+1], (SPEC_SIZE, -1))
    
    len_a = spec_a.shape[1]
    len_b = spec_b.shape[1]

    # find bigger spec
    spec_s = spec_a
    spec_l = spec_b

    if len_a>len_b:
        spec_s = spec_b
        spec_l = spec_a
        
    return spec_s, spec_l

@numba.njit()
def spec_dist(a,b, size):

    dist = (np.sum(np.abs(np.subtract(a, b)))) / size # manhattan
    
    #dist = (np.sum(np.subtract(a, b)*np.subtract(a, b))) / size # mean squared error
    #dist = np.sqrt((np.sum(np.subtract(a, b)*np.subtract(a, b))) / size)
    return dist
    

@numba.njit()
def calc_pairwise_pad(a, b):
    
    spec_s, spec_l = unpack_specs(a,b)
    n_padding = int(spec_l.shape[1] - spec_s.shape[1])
    
    # pad the smaller spec (if necessary)
    if n_padding!=0:
        pad = np.full((SPEC_SIZE, n_padding), 0.0)
        spec_s_padded = np.concatenate((spec_s, pad), axis=1)
        spec_s_padded = spec_s_padded.astype(np.float64)
    else:
        spec_s_padded = spec_s.astype(np.float64)

    # compute distance

    spec_s_padded = np.reshape(spec_s_padded, (-1)).astype(np.float64)
    spec_l = np.reshape(spec_l, (-1)).astype(np.float64)
    size = spec_l.shape[0]
    

    dist = spec_dist(spec_s_padded, spec_l, size)
    
    return dist


In [155]:
@numba.njit()
def calc_overlap_only(a,b): 
    spec_s, spec_l = unpack_specs(a,b)
    
    #only use overlap section from longer spec
    spec_l = spec_l[:spec_s.shape[0],:spec_s.shape[1]]
    
    spec_s = spec_s.astype(np.float64)
    spec_l = spec_l.astype(np.float64)
    
    size = spec_s.shape[1]*spec_s.shape[0]   
    dist = spec_dist(spec_s, spec_l, size)
    
    return dist

@numba.njit()
def calc_timeshift(a,b):
    
    spec_s, spec_l = unpack_specs(a,b)  
    len_l = spec_l.shape[1]
    len_s = spec_s.shape[1]


    # define start position
    min_overlap_frames = int(MIN_OVERLAP * len_s)
    start_timeline = min_overlap_frames-len_s
    max_timeline = len_l - min_overlap_frames
    
    n_of_calculations = (max_timeline+1-start_timeline)+(max_timeline+1-start_timeline)

    distances = np.full((n_of_calculations),3.)

    count=0
    
    for timeline_p in range(start_timeline, max_timeline+1):
        # mismatch on left side
        if timeline_p < 0:
            start_col_l = 0
            len_overlap = len_s - abs(timeline_p)

            end_col_l = start_col_l + len_overlap

            end_col_s = len_s # until the end
            start_col_s = end_col_s - len_overlap

        # mismatch on right side
        elif timeline_p > (len_l-len_s):
            start_col_l = timeline_p
            len_overlap = len_l - timeline_p
            end_col_l = len_l

            start_col_s = 0
            end_col_s = start_col_s + len_overlap

        # no mismatch on either side
        else:
            start_col_l = timeline_p
            len_overlap = len_s
            end_col_l = start_col_l + len_overlap

            start_col_s = 0
            end_col_s = len_s # until the end
            

        s_s = spec_s[:,start_col_s:end_col_s].astype(np.float64)
        s_l = spec_l[:,start_col_l:end_col_l].astype(np.float64)
        
        size = s_s.shape[0]*s_s.shape[1]
        distances[count] = spec_dist(s_s, s_l, size)

        count = count + 1
    
    min_dist = np.min(distances)
                                                     
    return min_dist



@numba.njit()
def calc_timeshift_pad(a,b):
    
    spec_s, spec_l = unpack_specs(a,b)
    
    len_s = spec_s.shape[1]
    len_l = spec_l.shape[1]

    nfreq = spec_s.shape[0] 

    # define start position
    min_overlap_frames = int(MIN_OVERLAP * len_s)
    start_timeline = min_overlap_frames-len_s
    max_timeline = len_l - min_overlap_frames
    
    n_of_calculations = int((((max_timeline+1-start_timeline)+(max_timeline+1-start_timeline))/2) +1)

    distances = np.full((n_of_calculations),999.)

    count=0
    
    for timeline_p in range(start_timeline, max_timeline+1,2):
        #print("timeline: ", timeline_p)
        # mismatch on left side
        if timeline_p < 0:

            len_overlap = len_s - abs(timeline_p)
            
            pad_s = np.full((nfreq, (len_l-len_overlap)),0.)
            pad_l = np.full((nfreq, (len_s-len_overlap)),0.)

            s_config = np.append(spec_s, pad_s, axis=1).astype(np.float64)
            l_config = np.append(pad_l, spec_l, axis=1).astype(np.float64)

        # mismatch on right side
        elif timeline_p > (len_l-len_s):
            
            len_overlap = len_l - timeline_p

            pad_s = np.full((nfreq, (len_l-len_overlap)),0.)
            pad_l = np.full((nfreq, (len_s-len_overlap)),0.)

            s_config = np.append(pad_s, spec_s, axis=1).astype(np.float64)
            l_config = np.append(spec_l, pad_l, axis=1).astype(np.float64)

        # no mismatch on either side
        else:
            len_overlap = len_s
            start_col_l = timeline_p
            end_col_l = start_col_l + len_overlap

            pad_s_left = np.full((nfreq, start_col_l),0.)
            pad_s_right = np.full((nfreq, (len_l - end_col_l)),0.)

            l_config = spec_l.astype(np.float64)
            s_config = np.append(pad_s_left, spec_s, axis=1).astype(np.float64)
            s_config = np.append(s_config, pad_s_right, axis=1).astype(np.float64)
        
        size = s_config.shape[0]*s_config.shape[1]
        distances[count] = spec_dist(s_config, l_config, size)
        count = count + 1


    min_dist = np.min(distances)
    return min_dist

In [156]:
set_defaults()
#duration_method_types = ['pad', 'stretch', 'pairwise-pad', 'overlap-only', 'timeshift-overlap', 'timeshift-pad']
duration_method_types = ['timeshift-pad']

# this is a tricky one, because for pad and stretch, I can use the default metric type from umap with varying 
# input, but for the other duration_method_types, I need to write a numba-compatible custom distance function
# thus the 3 "cases"

for duration_method in duration_method_types:
    
    outname = os.path.join(os.path.sep, OUT_COORDS, get_param_string())
    print(outname)
    
    # baseline pipeline
    if duration_method=='pad':
        specs = spec_df.spectrograms
        specs = [calc_zscore(x) for x in specs]
        data = create_padded_data(specs)
        calc_umap(data, outname)
    
    # using stretched specs as input
    elif duration_method=='stretch':
        specs = spec_df.stretched_spectrograms
        specs = [calc_zscore(x) for x in specs]
        data = create_padded_data(specs)
        calc_umap(data, outname)
        
    # pairwise-pad, overlap-only, timeshift-overlap and timeshift-pad all require
    # custom distance functions
    else:
        specs = spec_df.spectrograms
        specs = [calc_zscore(x) for x in specs]
        
        maxlen= np.max([spec.shape[1] for spec in specs]) * SPEC_SIZE +1
        trans_specs = [pad_transform_spectro(spec, maxlen) for spec in specs]
        data = np.asarray(trans_specs)

        metric_dict = {'pairwise-pad': calc_pairwise_pad,
                      'overlap-only': calc_overlap_only,
                      'timeshift-overlap': calc_timeshift,
                      'timeshift-pad': calc_timeshift_pad}
        
        calc_umap(data, outname, metric=metric_dict[duration_method])
    

/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_timeshift-pad_0_1_15_5


/home/mthomas/anaconda3/envs/meerkat_umap_env_3/lib/python3.7/site-packages/umap/umap_.py:1728: UserWarning: custom distance metric does not return gradient; inverse_transform will be unavailable. To enable using inverse_transform method method, define a distance function that returns a tuple of (distance [float], gradient [np.array])
  "custom distance metric does not return gradient; inverse_transform will be unavailable. "
/home/mthomas/anaconda3/envs/meerkat_umap_env_3/lib/python3.7/site-packages/umap/umap_.py:1728: UserWarning: custom distance metric does not return gradient; inverse_transform will be unavailable. To enable using inverse_transform method method, define a distance function that returns a tuple of (distance [float], gradient [np.array])
  "custom distance metric does not return gradient; inverse_transform will be unavailable. "
/home/mthomas/anaconda3/envs/meerkat_umap_env_3/lib/python3.7/site-packages/umap/umap_.py:1728: UserWarning: custom distance metric does not

## Different distance metrics

In [9]:
set_defaults()
    
metric_types = ['euclidean', 'manhattan', 'cosine', 'correlation']

specs = spec_df.spectrograms
specs = [calc_zscore(x) for x in specs]
data = create_padded_data(specs)

for metric_type in metric_types:
    
    outname = os.path.join(os.path.sep, OUT_COORDS, get_param_string())
    print(outname)
    calc_umap(data, outname, metric=metric_type)

/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_euclidean_pad_0_1_15_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_15_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_cosine_pad_0_1_15_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_correlation_pad_0_1_15_5_melspecs


## Different UMAP params

In [10]:
set_defaults()

n_neighbors_types = [5,15,30,50,100,150,200]

specs = spec_df.spectrograms
specs = [calc_zscore(x) for x in specs]
data = create_padded_data(specs)

for n_neighbors in n_neighbors_types:
    outname = os.path.join(os.path.sep, OUT_COORDS, get_param_string())
    print(outname)
    calc_umap(data, outname, n_neighbors=n_neighbors)

/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_5_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_15_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_30_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_50_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_100_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/interim/parameter_search/umap_coords/zs_manhattan_pad_0_1_150_5_melspecs
/home/mthomas/Documents/MPI_work/projects/meerkat/meerkat_umap/meerkat_umap/data/